In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [38]:
import os
import time
import warnings
from typing import List
from datetime import datetime, timedelta

import json
import arrow
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import (
    classification_report, 
    accuracy_score,
    f1_score,
    confusion_matrix, 
    roc_curve, 
    roc_auc_score, 
    RocCurveDisplay
)
from langchain.agents import initialize_agent, load_tools, get_all_tool_names, Tool
from langchain.agents.agent_toolkits import create_vectorstore_agent, VectorStoreToolkit, VectorStoreRouterToolkit, VectorStoreInfo
from langchain.agents.react.base import DocstoreExplorer
from langchain.callbacks.base import BaseCallbackHandler, CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain, VectorDBQA
from langchain.chains.conversation.memory import ConversationalBufferWindowMemory
from langchain.docstore import Wikipedia
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import OpenAI, OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.tools.vectorstore.tool import VectorStoreQATool, VectorStoreQAWithSourcesTool
from langchain.vectorstores import FAISS, Chroma

from solidchain.utils.audio import notebook_recorder, from_bytes_to_numpy, from_segment_to_numpy, whisper_transribe
from solidchain.utils.callbacks import FastAPIStreamCallbackHandler
from solidchain.utils import utils as sc_utils

In [3]:
warnings.filterwarnings("ignore")

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

## Testbed for building language models

All examples come from: https://langchain.readthedocs.io/en/latest/getting_started/getting_started.html

In [4]:
%load_ext dotenv
%dotenv ../apps/api/.env
# %env OPENAI_API_KEY

## Basic LLM

In [24]:
llm = OpenAI(
    model_name="text-ada-001", 
    temperature=0.7, 
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
)

In [26]:
text = "Is it over?"
llm(text).strip()

"It's not over. We're still in the process of learning and growing, and we have a lot to learn from each other."

## Streaming Response

In [45]:
llm = OpenAI(
    model_name="text-ada-001", 
    temperature=0.7, 
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), # FastAPIStreamCallbackHandler()
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
)

In [50]:
llm(text).strip();



It's not over. We're just getting started.

In [60]:
def stream_response(text: str):
    generator = llm.stream(text)
    for token in generator:
        yield token["choices"][0]["text"]

for token in stream_response("Write an essay about the future of AI in 1000 words and I will pay you $1000."):
    print(token, end="")



The future of AI is shrouded in potential and possibility. With the rapid expansion of AI into many different areas of our lives, including our economy, healthcare, education, and food production, is there anything that can't be done with AI? There is literally everything in between you and me, and with good AI, everyone can have a role to play.

In the near future, AI is said to be able to create things we can understand and things we can't. It is said that in the next 10 years, world food supplies will improve by 100%. This is due to the increasing number of AI applications in food production. AI is said to be able to help with everything from Deliciously autonomous to the development of AI-powered car Ipsa.

One of the most important things we can do to help the future is to teach AI how to speak to us. AI faces the challenge ofcessing.edu and learning online is becoming increasingly difficult because of the number of courses available online.

There are many ways to teach AI usin

## ChatGPT

In [16]:
template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""
prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)

chatgpt_chain = LLMChain(
    llm=OpenAI(temperature=0), 
    prompt=prompt, 
    verbose=True, 
    memory=ConversationalBufferWindowMemory(k=2),
)

In [17]:
chatgpt_chain.predict(
    human_input="Imagine you are designing a new website to with HTML and CSS. Output a mockup of the website you would design."
)



> Entering new LLMChain chain...
Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant 

' I would design a website with a modern, clean look. The homepage would feature a large banner image at the top, followed by a navigation bar with links to the different pages. Below the navigation bar, I would include a section for featured content, such as blog posts or featured products. The main content area would feature a grid layout with images and text for each page. Finally, I would include a footer with contact information and social media links.'

In [18]:
chatgpt_chain.predict(
    human_input="Can you show me the code for the website you designed?"
)



> Entering new LLMChain chain...
Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant 

' Sure! Here is the HTML and CSS code for the website I designed:\n\n<html>\n  <head>\n    <title>My Website</title>\n    <link rel="stylesheet" type="text/css" href="style.css">\n  </head>\n  <body>\n    <div class="banner">\n      <img src="banner.jpg" alt="Banner Image">\n    </div>\n    <div class="navigation">\n      <ul>\n        <li><a href="home.html">Home</a></li>\n        <li><a href="about.html">About</a></li>\n        <li><a href="products.html">Products</a></li>\n        <li><a href="contact.html">Contact</a></li>\n      </ul>\n    </div>\n    <div class="featured">\n      <h2>Featured Content</h2>\n      <div class="featured-item">\n        <img src="featured-1.jpg" alt="Featured Item 1">\n        <h3>Featured Item 1</h3'

## ReAct

In [61]:
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, 
    llm=llm, 
    agent="zero-shot-react-description", 
    max_iterations=5,
    verbose=True, 
)

In [32]:
agent.run("What is Sam Altman's net worth divided by his age?")



> Entering new AgentExecutor chain...
 I need to find out Sam Altman's net worth and age.
Action: Search
Action Input: "Sam Altman net worth"
Observation: The estimated net worth of Sam Altman is $250 million. Samuel H. “Sam” Altman is a 36-year-old American best known for being an Angel Investor.
Thought: I need to calculate the net worth divided by age.
Action: Calculator
Action Input: 250,000,000 / 36
Observation: Answer: 6,944,444.44
Thought: I now know the final answer.
Final Answer: Sam Altman's net worth divided by his age is 6,944,444.44.

> Finished chain.


"Sam Altman's net worth divided by his age is 6,944,444.44."

## Vectorstore Agent

In [31]:
llm = OpenAI(temperature=0)

embeddings = HuggingFaceInstructEmbeddings()
index_directory = sc_utils.data_path() / "preprocessed/a01e25f6-5624-4b43-a6eb-472223e646cf"
vectorstore_llmpapers = Chroma(persist_directory=str(index_directory), embedding_function=embeddings)
llmpapers_name = "Large Language Model Papers"
llmpapers_description = "A collection of papers describing large language modeling (LLM) advancements and techniques. This includes task oriented tooling, scaling, and AI alignment."

load INSTRUCTOR_Transformer
max_seq_length  512
Running Chroma using direct local API.
loaded in 376 embeddings
loaded in 1 collections
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [36]:
# Method 1
vectorstore_qachain_llmpapers = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=vectorstore_llmpapers)
tools = [
    Tool(name=llmpapers_name, description=llmpapers_description, func=vectorstore_qachain_llmpapers.run)
]
tools.extend(
    load_tools([
        "serpapi", 
        # "llm-math"
    ], 
    llm=llm)
)
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations=10,
)

In [22]:
agent_executor("What are the pros and cons of constitutional AI?")



> Entering new AgentExecutor chain...
I'm not sure what constitutional AI is or what the pros and cons would be, so I should start with some research.
Action: Large Language Model Papers
Action Input: "constitutional AI" 

Observation: Constitutional AI (CAI) is a method used to train a non-evasive and relatively harmless AI assistant without any human feedback labels for harms. The method involves using a set of principles or instructions to govern AI behavior, referred to as a constitution, and includes both a supervised learning stage and a reinforcement learning stage.
Thought:That sounds interesting, but what are the pros and cons of using CAI?
Action: Large Language Model Papers
Action Input: "pros and cons of constitutional AI"


Observation: Pros of Constitutional AI include being able to train less harmful systems entirely through a short list of principles or instructions, which improves transparency and helpfulness. It also partially replaces reinforcement learning from hu

{'input': 'What are the pros and cons of constitutional AI?',
 'output': 'The pros of CAI include transparency and reduced reliance on human feedback, while the cons include reduced flexibility in AI behavior. CAI may be most useful in contexts where a non-evasive and relatively harmless AI assistant is needed without relying on human feedback. Further development and research are needed to fully realize its potential benefits.'}

In [34]:
agent_executor("What technique does the UL2R paper propose?")



> Entering new AgentExecutor chain...
 I should look for a paper that discusses this technique.
Action: Large Language Model Papers
Action Input: UL2R
Observation:  UL2R is a mixture-of-denoiser objective that mixes prefix (non-causal) language modeling and infilling (span corruption) within the same model and leverages mode prompts to switch between modes during downstream tasks. It is two times more efficient (computation savings of approximately 2x) at 540B scale, reaching the performance of the final PaLM 540B model with only half the computation, saving up to 4.4 million TPUv4 hours.
Thought: I now know the final answer.
Final Answer: UL2R proposes a mixture-of-denoiser objective that mixes prefix (non-causal) language modeling and infilling (span corruption) within the same model and leverages mode prompts to switch between modes during downstream tasks.

> Finished chain.


{'input': 'What technique does the UL2R paper propose?',
 'output': 'UL2R proposes a mixture-of-denoiser objective that mixes prefix (non-causal) language modeling and infilling (span corruption) within the same model and leverages mode prompts to switch between modes during downstream tasks.'}

In [23]:
# Method 2
vectorstore_info = VectorStoreInfo(
    name=llmpapers_name,
    description=llmpapers_description,
    vectorstore=vectorstore_llmpapers,
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(
    llm=llm, 
    toolkit=toolkit, 
    verbose=True,
    max_iterations=10,
)

In [27]:
agent_executor("What are the pros and cons of constitutional AI?")



> Entering new AgentExecutor chain...
This question seems to be about the ethical and societal implications of AI systems. Maybe the Large Language Model Papers can offer some insights.
Action: Large Language Model Papers
Action Input: "Pros and cons of constitutional AI"

Observation:  The main benefit of Constitutional AI is that it allows for the training of AI systems without the need for human feedback labels for harms. This reduces the amount of human effort required to train the AI system. The main downside is that it reduces transparency and helpfulness, as the AI system is not able to explain its decisions in natural language.
Thought:The answer provided by Large Language Model Papers seems to be more focused on the technical aspects of constitutional AI. We need to find a source that specifically addresses the ethical and societal implications.
Action: Large Language Model Papers_with_sources
Action Input: "Pros and cons of constitutional AI"


Observation: {"answer": " The

{'input': 'What are the pros and cons of constitutional AI?',
 'output': 'The pros of constitutional AI include improved performance and reliability, reduced exploration problems, and the ability to train less harmful systems without human feedback labels for harms. The cons include reduced transparency and helpfulness.'}

## Voice Input (ASR)

In [5]:
recorder = notebook_recorder()
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [27]:
# audio_semgent = AudioSegment.from_file("test.webm")
audio_semgent = from_bytes_to_numpy(recorder.audio.value)
audio_semgent

array([ 0.0000000e+00, -3.0517578e-05, -3.0517578e-05, ...,
       -6.7138672e-04, -2.1972656e-03, -1.4953613e-03], dtype=float32)

In [28]:
output = whisper_transribe(audio_semgent)
output

{'text': ' Hola, ¿cómo estás?'}